In [ ]:
#import scipy.spatial.distance as sdist
import umap
from sklearn import preprocessing
import time
import pandas as pd
import numpy as np
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree, fcluster
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
#import math
import os
import random
#import requests
#from BeautifulSoup import BeautifulSoup
import matplotlib.lines as mlines
import matplotlib as mpl
from pylab import *
#from ete3 import NCBITaxa
#ncbi = NCBITaxa()
import json
from ast import literal_eval
from matplotlib.cm import ScalarMappable
import scipy
import scipy.stats
import scipy.signal
from matplotlib.ticker import MaxNLocator
strain_color_list = ["#33bbff","#0050e6","#009999", "#777777"]
umap_plot_shape = ['D','o','s','v']
default_color_list = [u'#1f77b4', u'#ff7f0e', u'#2ca02c', u'#d62728', u'#9467bd', u'#8c564b', u'#e377c2', u'#7f7f7f', u'#bcbd22', u'#17becf']

In [ ]:
# function 
# input is query_cluster, qseqid, qstart and qend
# output is to print out corresponding read
def get_sequence(assembly_dirt, assembly_file_name, qseqid, qstart, qend):
    with open(assembly_dirt + assembly_file_name) as finput:
        while True:
            line_read = finput.readline()  
            if len (line_read) == 0:
                break
            if ">" in line_read:
                if qseqid in line_read:
                    line_read = ''
                    while True:
                        line_read_temp = finput.readline().strip()
                        line_read = line_read+line_read_temp
                        if len (line_read) == 0 or ">" in line_read:
                            break
                    string_of_mobile_gene = line_read.strip()[qstart-1:qend]
                    #print len(string_of_mobile_gene),string_of_mobile_gene
                    return string_of_mobile_gene
                    break

In [ ]:
# a function to take intervals and return merged intervals
def merge_intervals(intervals):
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []

    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            # test for intersection between lower and higher:
            # we know via sorting that lower[0] <= higher[0]
            if higher[0] <= lower[1]:
                upper_bound = max(lower[1], higher[1])
                merged[-1] = (lower[0], upper_bound)  # replace by merged interval
            else:
                merged.append(higher)
    return merged

In [ ]:
# this is the function to process data from variant calling from bcftools
# inputs are name of vcf file, dirt and the two criteria to remove SNPs
# 1st is the least number of cells that cover this SNP, 
# it is the cutoff_cell_count, default is 0.05*cell_number
# 2nd is the cutoff to remove SNPs that either reference or alter is covered less than this nubmer of cells
    # it is the larger value of (cutoff_type_count, cell_number*cutoff_cell_ratio)
# Another input is the cutoff_SNP_ratio (ratio of SNPs to cover), 
# if a cell covers less than this ratio of SNPs, drop this cell
# output files are: 
    # a dataframe of all SNP information
    # a dataframe with less covered snps removed, and
    # a dataframe with less covered SNPs removed and cells with less coverage removed
# a plot of the SNP ratio
def strain_split(file_name = '',work_dirt_temp = '', cutoff_cell_ratio=0.05, \
                 cutoff_type_count=2, cutoff_type_ratio=0.01,cutoff_SNP_ratio = 0.01):
    # this is to parse vcf SNP file into lists
    barcode_list = []
    contig_list = []
    location_on_contig_list = []
    reference_list = []
    alter_list = []
    sequence_list = [] # the sequence at this location for this barcode
    # it is a list of lists, each list element is a vector of all reads from this SNP, length is cell number
    # if not mapped, give it 0
    # if only 1 read covers this location, give it 0
    # if less than 99% have the same snp call, give it 0
    # same as reference is 1
    # not the same as reference is -1
    total_line = 0
    with open(work_dirt_temp + file_name) as finput:
        while True:
            line_read = finput.readline()
            if len (line_read) == 0:
                break
            # First parse and construct barcode list, and assign cutoff values
            total_line += 1
            if total_line == 4:
                parsed_bam = line_read.split()
                for i in range(len(parsed_bam)):
                    barcode_with_bam = parsed_bam[i]
                    if '.bam' == barcode_with_bam[-4:]:
                        barcode_list += [barcode_with_bam[:-4]]
                cutoff_cell_count = int(cutoff_cell_ratio*len(barcode_list))
                cutoff_type_count = max(cutoff_type_count, \
                                       int(cutoff_type_ratio*len(barcode_list)))
                
            #print cutoff_cell_count
            if line_read[0] != '#': # all information part starts with '#', so this is a line with mapping info
                line_split_temp = line_read.split()
                #print line_split_temp
                contig_list += [int(line_split_temp[0].split('_')[1])]
                location_on_contig_list += [int(line_split_temp[1])]
                reference_list += [line_split_temp[3]]
                alter_list += [line_split_temp[4]]
                mapping_count_list_temp = [] # this is a temp count list for this SNP
                # this is to assign value of each cell for each SNP
                # if a cell has only 1 read for a SNP or less than 99% sure, assign 0
                for i in range(9, len(line_split_temp)):
                    mapping_result_temp = line_split_temp[i].split(":")[-1]
                    reference_count_temp = int(mapping_result_temp.split(',')[0])
                    alter_count_temp = int(mapping_result_temp.split(',')[1])
                    total_count_temp = reference_count_temp + alter_count_temp
                    if total_count_temp < 2:
                        mapping_count_list_temp += [0]
                    elif max(reference_count_temp, alter_count_temp)/float(total_count_temp) < 0.99:
                        mapping_count_list_temp += [0]
                    else:
                        if max(reference_count_temp, alter_count_temp) == reference_count_temp:
                            mapping_count_list_temp += [1]
                        elif max(reference_count_temp, alter_count_temp) == alter_count_temp:
                            mapping_count_list_temp += [-1]
                        else:
                            print "something wrong"
                sequence_list += [mapping_count_list_temp]
                
    snp_pd = pd.DataFrame({'1': sequence_list[0]})
    for i in range(len(sequence_list)):
        snp_pd[str(i+1)] = sequence_list[i]
    snp_pd = snp_pd.T
    snp_pd.columns = barcode_list
    snp_pd['contig_list'] = contig_list
    snp_pd['location_on_contig_list'] = location_on_contig_list
    snp_pd['reference_list'] = reference_list
    snp_pd['alter_list'] = alter_list
    snp_pd.to_csv(work_dirt_temp + file_name + '_snp.csv',index=False)
    # this is the parsed SNP dataframe, 
    # 0 means no coverage, 
    # 1 means same as reference, 
    # -1 means same as alternate sequence
    
    # drop those alleles that have less than cutoff location counted as mutation
    snp_pd = pd.read_csv(work_dirt_temp + file_name + '_snp.csv')
    
    to_drop_row_list = []
    for i in range(len(snp_pd)):
        if sum(snp_pd.iloc[i][:-4]!=0) <cutoff_cell_count:
            to_drop_row_list.append(i)
            continue
        if sum(snp_pd.iloc[i][:-4]==-1) <cutoff_type_count or sum(snp_pd.iloc[i][:-4]==1) <cutoff_type_count :
            to_drop_row_list.append(i)
    print "File name is {}, total SNPs before drop is {}, SNPs to drop is {}."\
    .format(file_name, len(snp_pd), len(to_drop_row_list))
    
    snp_pd = snp_pd.drop(to_drop_row_list, axis=0)
    snp_pd = snp_pd.reset_index(drop=True)
    snp_pd.to_csv(work_dirt_temp + file_name + '_snp_filtered.csv',index=False)
    # this is the dataframe with less covered SNPs removed
    
    # This is to remove cells with less SNPs covered
    snp_pd = pd.read_csv(work_dirt_temp + file_name + '_snp_filtered.csv')
    cutoff_SNP_count = max(cutoff_SNP_ratio * len(snp_pd), 10)
    to_drop_column_list = []
    barcode_list = list(snp_pd.columns.values[:-4])
    for i in range(snp_pd.shape[1]-4):
        if sum(snp_pd[barcode_list[i]]!=0)< cutoff_SNP_count:
             to_drop_column_list += [barcode_list[i]]
                
    print "Total cells before drop is {}, cells to drop is {}."\
    .format(snp_pd.shape[1]-4, len(to_drop_column_list))
    
    snp_pd = snp_pd.drop(to_drop_column_list, axis=1)
    snp_pd.to_csv(work_dirt_temp + file_name + '_snp_filtered_bc_dropped.csv',index=False)    
    
    
    # this is to plot SNP ratio that fits to reference
    snp_pd = pd.read_csv(work_dirt_temp + file_name + '_snp_filtered_bc_dropped.csv')
    total_ratio_of_snps_list = []
    for i in range(len(snp_pd)):
        total_coverage_temp = sum(list(snp_pd.iloc[i][:-4]!=0))
        if total_coverage_temp == 0:
            print i
            total_ratio_of_snps_list+= [1]
            continue    
        total_ratio_of_snps_list += [sum(list(snp_pd.iloc[i][:-4]==1))/float(total_coverage_temp)]
    plt.figure()
    count_temp = plt.hist(np.array(total_ratio_of_snps_list),bins = np.arange(0,1.01,0.02))
    #plt.ylim(0,1.1*max(count_temp[0][:45]))
    plt.tick_params(axis='both', direction="in", which='both', top=True, right=True)
    plt.xlabel("Ratio of cells matching strain 1")
    plt.ylabel("Number of alleles")
    plt.savefig(work_dirt_temp+\
                file_name+'_SNPs.svg', format='svg',bbox_inches='tight')
    return(total_ratio_of_snps_list)

In [ ]:
work_dirt_temp = '/media/z/feb2021/microbe_seq_analysis/Q5_variant_calling_100fewer/raw_vcf_files/'
# this is the newest dirtecorty
file_list_temp = os.listdir(work_dirt_temp)
list_of_ratio_lists = []
for file_temp in file_list_temp:
    if ".vcf" == file_temp[-4:]:
        total_ratio_of_snps_list = strain_split(file_temp,work_dirt_temp = work_dirt_temp)
        list_of_ratio_lists.append(total_ratio_of_snps_list)

-----

In [45]:
def check_strain(file_name, tree_cut_threshold = 2.7):
    #file_name = vcf_files_list[file_index]#'59_calls_snps_qual30.vcf'
    print file_name
    work_dirt_temp = '/media/z/feb2021/microbe_seq_analysis/Q5_variant_calling_100fewer/raw_vcf_files/'
    snp_pd = pd.read_csv(work_dirt_temp + file_name + '_snp_filtered_bc_dropped.csv')
    print(len(snp_pd))
    snp_pd_temp = snp_pd
    snp_pd_temp = snp_pd_temp.drop(['contig_list','location_on_contig_list', 'reference_list', 'alter_list'], axis=1)
    snp_pd_temp = snp_pd_temp.T
    #X = snp_pd_temp
    X=preprocessing.normalize(snp_pd_temp)
    linked = linkage(X, method='ward')
    matplotlib.rcParams.update({'font.size': 50})
    fig, ax = plt.subplots(1, 1, figsize=(3,9))

    #ax.set_title('Hierarchical Clustering of Cells')
    #ax.set_ylabel('Distance')
    #ax.set_xlabel("Single cells")
    plt.yticks([0,50,100,150,200])
    plt.tick_params(axis='both', direction="in", which='both',right=True,length =15,width=3)
    matplotlib.rcParams['lines.linewidth'] = 0.8

    #hierarchy.set_link_color_palette(['#F16A70', '#4D4D4D', '#B1D877', '#8CDCDA',])
    strain_color_list_BV = [strain_color_list[1], strain_color_list[0], strain_color_list[2],strain_color_list[1]]
    hierarchy.set_link_color_palette(strain_color_list_BV)
    dendrogram_plot = dendrogram(linked,  
                                 orientation='left',
                                 distance_sort='ascending',
                                 leaf_font_size = 2,
                                 no_labels=True,
                                 color_threshold = tree_cut_threshold,
                                 above_threshold_color = 'black',
                                 show_leaf_counts=False)

    plt.xticks([])
    plt.gca().spines.values()[0].set_visible(False)
    plt.gca().spines.values()[1].set_visible(False)
    plt.gca().spines.values()[2].set_visible(False)
    plt.gca().spines.values()[3].set_visible(False)
    plt.savefig(file_name.split("_")[0]+'_SNP_similarity_tree.svg', format='svg',bbox_inches='tight')
    matplotlib.rcParams.update({'font.size': 12})
    
    plt.figure()
    X = snp_pd_temp
    X=preprocessing.normalize(snp_pd_temp)
    reducer = umap.UMAP(random_state=42)
    embedding = reducer.fit_transform(X)
    embedding.shape
    plt.scatter(embedding[:,0], embedding[:,1],s=3)
    plt.savefig(file_name.split("_")[0]+'_SNP_umap.svg', format='svg',bbox_inches='tight')
    # this is to construct a SNP similarity matrix from data
    # the sequence is based on the sequence of dendragram
    index_int_list = []
    for i in dendrogram_plot['ivl']:
        index_int_list.append(int(i))
    SNP_similarity_df = pd.DataFrame\
    (-1.0, index=snp_pd.columns[index_int_list], columns=snp_pd.columns[index_int_list])
    for i in range(len(SNP_similarity_df)):
        column_name_a = snp_pd.columns[index_int_list[i]]
        for j in range(len(SNP_similarity_df)):
            if i == j:
                SNP_similarity_df.iloc[i][j] = 1.0
                continue
            column_name_b = snp_pd.columns[index_int_list[j]]
            check_snp_temp = snp_pd[[column_name_a, column_name_b]]
            check_snp_temp = check_snp_temp.loc[check_snp_temp[column_name_b]!=0]
            check_snp_temp = check_snp_temp.loc[check_snp_temp[column_name_a]!=0]
            if len(check_snp_temp) == 0:# if there is no overlap between 2 cells, give it -1 
                continue
            SNP_similarity_df.iloc[i][j] = sum(check_snp_temp[column_name_b] == check_snp_temp[column_name_a])/\
            float(len(check_snp_temp))
    SNP_similarity_df.to_csv(work_dirt_temp+file_name.split("_")[0]+'_SNP_similarity_df.csv', index = False)    
    fig, ax = plt.subplots()
    data_color = []
    fig.set_size_inches(6,6, forward=True)
    my_cmap = plt.cm.get_cmap(matplotlib.cm.cmap_d.keys()[58])
    my_cmap = mpl.colors.ListedColormap(['w', default_color_list[4]])
    my_cmap = 'RdBu'
    maximum_similarity = 1
    SNP_similarity_df = pd.read_csv(work_dirt_temp+file_name.split("_")[0]+'_SNP_similarity_df.csv')
    ax.imshow(SNP_similarity_df, cmap=my_cmap, vmax=maximum_similarity, vmin=0)

    ax.set_xlabel("Single cells", fontsize=15)
    #ax.set_ylabel("Assembled genomes 1 to 52", fontsize=15)
    sm = ScalarMappable(cmap=my_cmap, norm=plt.Normalize(0,maximum_similarity))
    sm.set_array([])
    cbar = plt.colorbar(sm, orientation = 'vertical',fraction = 0.05, pad = 0.05, \
                        aspect = 15, ticks = [0, maximum_similarity])
    cbar.set_label('Similarity of SNPs between cells', rotation=90,labelpad=-1,fontsize=15)
    plt.savefig(file_name.split("_")[0]+'_SNP_similarity.svg', format='svg',bbox_inches='tight')
    return(snp_pd, linked, embedding)

----

In [ ]:
file_name_check = "17_calls_snps_qual30.vcf"
snp_pd, linked, embedding=check_strain(file_name=file_name_check, tree_cut_threshold=2)

In [ ]:
cluster_number = 3
cutree_contigs = cut_tree(linked, n_clusters=cluster_number)    
print len(cutree_contigs), sum(cutree_contigs==[0]),sum(cutree_contigs==[1]),sum(cutree_contigs==[2])

umap_compare = pd.DataFrame({'Barcode':snp_pd.columns[:-4]})
umap_compare['X'] = embedding[:,0]
umap_compare['Y'] = embedding[:,1]
cluster_hier = []
for i in range(len(cutree_contigs)):   
    cluster_hier.append(cutree_contigs[i][0])
cluster_umap = []
for i in range(len(umap_compare)):
    if umap_compare['X'][i]>0 and umap_compare['Y'][i]>1:
        cluster_umap.append(1)
    elif umap_compare['X'][i]<-5 and umap_compare['Y'][i]<-0.7:
        cluster_umap.append(2)
    elif umap_compare['X'][i]<-5 and umap_compare['Y'][i]>-0.7:
        cluster_umap.append(0)
   # else:
    #    cluster_umap.append('B')
umap_compare['cluster_umap'] = cluster_umap
umap_compare['cluster_hier'] = cluster_hier     
#print sum(umap_compare['cluster_umap'] == umap_compare['cluster_hier'])/float(len(umap_compare['cluster_umap']))
umap_compare.loc[umap_compare['cluster_umap'] != umap_compare['cluster_hier']]

In [ ]:
for j in range(cluster_number):    
    umap_compare_bin = umap_compare.loc[umap_compare['cluster_hier']==j]
    plt.scatter(umap_compare_bin['X'], umap_compare_bin['Y'],s=10,c=strain_color_list[j],marker=umap_plot_shape[j])
plt.xticks([])
plt.yticks([])
plt.ylabel('UMAP dimension 2')
plt.xlabel("UMAP dimension 1")
plt.savefig(file_name_check.split("_")[0]+'_SNP_umap_final.svg', format='svg',bbox_inches='tight')

In [ ]:
cluster_number = 3
cutree_contigs = cut_tree(linked, n_clusters=cluster_number)    
len(cutree_contigs)

In [ ]:
# This is to construct the genotype list of each strain

for i in range(cluster_number):
    index_of_this_cluster = [] 
    for j in range(len(cutree_contigs)):
        if cutree_contigs[j] == i:
            index_of_this_cluster += [j] 
    snp_pd['hierarchy_cluster_'+str(i)+'pos']=(snp_pd[snp_pd.columns.values[index_of_this_cluster]]==1).sum(axis=1)
    snp_pd['hierarchy_cluster_'+str(i)+'neg']=(snp_pd[snp_pd.columns.values[index_of_this_cluster]]==-1).sum(axis=1)
    
strain_X_genotype = [[] for i in range(cluster_number)]
for i in range(len(snp_pd)):
    for j in range(cluster_number):
        # generate genotype of each strain
        # at each SNP, if both have less than 2 cells, assign it 0, means not sure
        # if the max genotype is less than 90% total, assign 0
        # otherwise assign the major genotype
        if snp_pd['hierarchy_cluster_'+str(j)+'pos'][i] <= 1 and snp_pd['hierarchy_cluster_'+str(j)+'neg'][i] <=1:
            strain_X_genotype[j].append(0)
            continue
        elif snp_pd['hierarchy_cluster_'+str(j)+'pos'][i]!= 0 and snp_pd['hierarchy_cluster_'+str(j)+'neg'][i] != 0:
            if max(snp_pd['hierarchy_cluster_'+str(j)+'pos'][i],snp_pd['hierarchy_cluster_'+str(j)+'neg'][i])/\
            float(min(snp_pd['hierarchy_cluster_'+str(j)+'pos'][i],snp_pd['hierarchy_cluster_'+str(j)+'neg'][i]))\
            < 9:
                strain_X_genotype[j].append(0)
                continue                
        if snp_pd['hierarchy_cluster_'+str(j)+'pos'][i] > snp_pd['hierarchy_cluster_'+str(j)+'neg'][i]:
            strain_X_genotype[j].append(1)
        else:
            strain_X_genotype[j].append(-1)
    
for j in range(cluster_number):
    snp_pd['strain_' + str(j) + '_genotype'] = strain_X_genotype[j]

# drop SNPs without information (a SNP with both 1 and -1 is informative)
rows_to_drop = []
for i in range(len(snp_pd)):
    genotype_SNP_temp = []
    for j in range(cluster_number):
        genotype_SNP_temp.append(snp_pd['strain_' + str(j) + '_genotype'][i])
    if not (1 in genotype_SNP_temp and -1 in genotype_SNP_temp):
        rows_to_drop.append(i)
print "SNPs before drop is {}, SNPs to drop is {}.".format(len(snp_pd), len(rows_to_drop))
snp_pd = snp_pd.drop(rows_to_drop, axis=0)
snp_pd = snp_pd.reset_index(drop=True)

In [ ]:
# this is to generate strain list
list_of_ratio = [[] for i in range(cluster_number)]
barcode_list_strain = [[] for i in range(cluster_number+1)] # the last strain is those not sure
for i in range(snp_pd.shape[1]):
    column_name_temp = snp_pd.columns[i]
    if not column_name_temp.isdigit(): # this means that this is not a barcode name
        continue
    if not column_name_temp in snp_pd.columns:
        continue
    temp_df_check = snp_pd.loc[snp_pd[column_name_temp]!=0]
    for j in range(cluster_number):
        temp_df_check_strain = temp_df_check.loc[temp_df_check['strain_' + str(j) + '_genotype']!=0]
        total_count_same = sum(temp_df_check_strain[column_name_temp] == \
        temp_df_check_strain['strain_' + str(j) + '_genotype'])
        list_of_ratio[j].append(total_count_same/float(len(temp_df_check_strain[column_name_temp])))
    list_of_ratio_check = [list_of_ratio[j][-1] for j in range(cluster_number)]
    if max(list_of_ratio_check) < 0.95:
        barcode_list_strain[-1].append(column_name_temp)
    elif sum(np.array(list_of_ratio_check)>0.95) > 1:
        print "Not sure, more than two strains match", column_name_temp
        barcode_list_strain[-1].append(column_name_temp)
    else:
        barcode_list_strain[list_of_ratio_check.index(max(list_of_ratio_check))].append(column_name_temp)
for i in range(len(barcode_list_strain)):
    print i, len(barcode_list_strain[i])

In [ ]:
# this is to count cell number in these 7 time points
time_points = [14,52,111,150,171,172,224] # this is in sample ID
# in days
time_points = [15,104,276,367,404,405,526] # this is in days
time_point_index_convcert = {14:0,
                             31:1,
                             32:1,
                             34:1,
                             8:2,
                             9:2,
                             5:3,
                             7:3,
                             15:3,
                             27:3,
                             10:4,
                             11:5,
                             12:5,
                             33:6,
                             35:6
                            }
number_of_strains = [[0]*7 for i in range(cluster_number)]
for i in range(cluster_number):
    for j in range(len(barcode_list_strain[i])): 
        barcode_temp = int(barcode_list_strain[i][j])
        time_point_index_temp = time_point_index_convcert[barcode_temp/100000]
        number_of_strains[i][time_point_index_temp] += 1
total_cells_known = [0] * 7
for i in range(len(time_points)):
    for j in range(cluster_number):
        total_cells_known[i] += float(number_of_strains[j][i])
        
color_1 = [177.0/255, 216.0/255, 119.0/255]
color_2 = [77.0/255, 77.0/255, 77.0/255]
color_3 = [241.0/255, 106.0/255, 112.0/255]
color_4 = [140.0/255, 220.0/255, 218.0/255]
color_list = [color_1]+[color_2]+[color_3]+[color_4]
marker_list = ['>', 'D', 's', 'o']

matplotlib.rcParams.update({'font.size': 15})
fig, ax = plt.subplots()
for i in range(cluster_number):
    ax.plot(time_points,  np.array(number_of_strains[i])/np.array(total_cells_known),\
            color=color_list[i],linewidth=2)
    ax.scatter(time_points, np.array(number_of_strains[i])/np.array(total_cells_known),\
               color=color_list[i],marker=marker_list[i], s = 35)
plt.xlabel("Time (days)")
plt.ylabel("Fraction of strains")
plt.ylim(-0.1,1.1)

plt.xticks(time_points)
ax.set_xticklabels(['15','104','276','367','','     405','526'])

strain_1_legend = mlines.Line2D([], [], color=color_list[0], marker=marker_list[0], linestyle='-',
                          markersize=7, label="strain 1", linewidth=2)
strain_2_legend = mlines.Line2D([], [], color=color_list[1], marker=marker_list[1], linestyle='-',
                          markersize=7, label="strain 2", linewidth=2)
strain_3_legend = mlines.Line2D([], [], color=color_list[2], marker=marker_list[2], linestyle='-',
                          markersize=7, label="strain 3", linewidth=2)
strain_4_legend = mlines.Line2D([], [], color=color_list[3], marker=marker_list[3], linestyle='-',
                          markersize=7, label="strain 4", linewidth=2)
#plt.legend(handles=[strain_0_legend, strain_1_legend, strain_2_legend, strain_3_legend], \
#           loc='upper left', fontsize=12, framealpha=0.5)
plt.tick_params(axis='both', direction="in", which='both', top=True, right=True)
legend1 = plt.legend(handles=[strain_1_legend,strain_2_legend,strain_3_legend], \
           loc='best', fontsize=12, framealpha=0.5)
#plt.legend(handles=[strain_3_legend, strain_4_legend], \
#           loc='upper center', fontsize=12, framealpha=0.5)
plt.gca().add_artist(legend1)
plt.savefig(file_name_check.split("_")[0]+'_time_plot.svg', format='svg',bbox_inches='tight')

In [ ]:
dirt_fastq = "/media/z/feb2021/04082019Nextseq/fastq_SE/"
strain_assembly_dirt = "/media/z/feb2021/microbe_seq_analysis/Q5_variant_calling_100fewer/strain_assembly/"
for strain_to_check in range(cluster_number):
    assembly_dirt_temp = strain_assembly_dirt + file_name_check.split('_')[0]+'_'+str(strain_to_check)+'/'
    os.mkdir(assembly_dirt_temp)
    for bc_temp in barcode_list_strain[strain_to_check]:
        !cp {dirt_fastq}{bc_temp}.fastq {assembly_dirt_temp}
    !cat {assembly_dirt_temp}*.fastq > {assembly_dirt_temp}all.fq
    !rm {assembly_dirt_temp}*.fastq
    !(spades.py --sc --careful --pe1-s {assembly_dirt_temp}all.fq -o {assembly_dirt_temp}all_scCareful) \
    1> {strain_assembly_dirt}/{file_name_check.split('_')[0]}_{str(strain_to_check)}.txt
    !cp {assembly_dirt_temp}all_scCareful/contigs.fasta \
    {strain_assembly_dirt}/{file_name_check.split('_')[0]}_{str(strain_to_check)}.fasta
    !rm -r {assembly_dirt_temp}